# Lesson 5: Creating an MCP Client 

이전 레슨에서는 2개의 tool을 노출하는 MCP research server를 생성했습니다. 이 레슨에서는 chatbot이 MCP client를 통해 server와 통신하도록 만들 것입니다. 이렇게 하면 chatbot이 MCP compatible하게 됩니다. 레슨 4에서 중단한 지점부터 계속할 것입니다. 즉, `research_server.py` file이 포함된 `mcp_project` folder가 다시 제공됩니다. 여기에 MCP chatbot file을 추가하고 environment를 업데이트할 것입니다.

## Back to the Chatbot Example

다음은 레슨 3의 chatbot 예제에서 가져온 주요 code 부분들(`process_query`, `chat_loop`)입니다. tool 정의와 실행의 부담이 이제 MCP server로 이전되었으므로, chatbot logic에는 사용자 query 처리와 사용자가 `quit`을 입력할 때까지 chat loop을 계속 실행하는 것과 관련된 code만 포함되어 있다는 점을 주목하세요.

In [1]:
from dotenv import load_dotenv
from google import genai
from google.genai import types
import os

load_dotenv()

# Set the API key from environment variable
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', '')

client = genai.Client()

def process_query(query):
    config = types.GenerateContentConfig(tools=[tools])
    
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=query,
        config=config
    )
    
    process_query = True
    while process_query:
        
        for part in response.candidates[0].content.parts:
            if hasattr(part, 'text') and part.text:
                print(part.text)
                process_query = False
                break
            
            elif hasattr(part, 'function_call') and part.function_call:
                
                function_call = part.function_call
                tool_name = function_call.name
                tool_args = dict(function_call.args)
                
                print(f"Calling tool {tool_name} with args {tool_args}")
                
                result = execute_tool(tool_name, tool_args)
                
                # Continue conversation with function response
                function_response = types.FunctionResponse(
                    name=tool_name, 
                    response={"result": result}
                )
                
                response = client.models.generate_content(
                    model='gemini-2.5-flash',
                    contents=[
                        types.Content(role='user', parts=[types.Part(text=query)]),
                        types.Content(role='model', parts=[types.Part(function_call=function_call)]),
                        types.Content(role='function', parts=[types.Part(function_response=function_response)])
                    ],
                    config=config
                )
                
                # Print the final response
                if response.candidates[0].content.parts:
                    for final_part in response.candidates[0].content.parts:
                        if hasattr(final_part, 'text') and final_part.text:
                            print(final_part.text)
                            break
                process_query = False


def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("\nQuery: ").strip()
            if query.lower() == 'quit':
                break
    
            process_query(query)
            print("\n")
        except Exception as e:
            print(f"\nError: {str(e)}")

## Building your MCP Client

이제 `process_query`와 `chat_loop` function들을 가져와서 `MCP_ChatBot` class로 래핑할 것입니다. chatbot이 server와 통신할 수 있도록 하기 위해, MCP client를 통해 server에 연결하는 method를 추가할 것입니다. 이는 다음 reference code에 제시된 구조를 따릅니다:

### Reference Code
``` python
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client

# Create server parameters for stdio connection
server_params = StdioServerParameters(
    command="uv",  # Executable
    args=["run example_server.py"],  # Command line arguments
    env=None,  # Optional environment variables
)

async def run():
    # Launch the server as a subprocess & returns the read and write streams
    # read: the stream that the client will use to read msgs from the server
    # write: the stream that client will use to write msgs to the server
    async with stdio_client(server_params) as (read, write): 
        # the client session is used to initiate the connection 
        # and send requests to server 
        async with ClientSession(read, write) as session:
            # Initialize the connection (1:1 connection with the server)
            await session.initialize()

            # List available tools
            tools = await session.list_tools()

            # will call the chat_loop here
            # ....
            
            # Call a tool: this will be in the process_query method
            result = await session.call_tool("tool-name", arguments={"arg1": "value"})


if __name__ == "__main__":
    asyncio.run(run())
`````

### Adding MCP Client to the Chatbot

MCP_ChatBot class는 다음 method들로 구성됩니다:
- `process_query`
- `chat_loop`
- `connect_to_server_and_run`
  
그리고 다음 attribute들을 가집니다:
- session (ClientSession type)
- anthropic: Anthropic                           
- available_tools

`connect_to_server_and_run`에서 client는 server를 실행하고 server가 제공하는 tool들의 list를 요청합니다(client session을 통해). tool 정의들은 `available_tools` variable에 저장되고 `process_query`에서 LLM으로 전달됩니다.

<img src="images/tools_discovery.png" width="400">


`process_query`에서 LLM이 tool 실행이 필요하다고 판단하면, client session이 server에 tool call request를 전송합니다. 반환된 response는 LLM으로 전달됩니다. 

<img src="images/tool_invocation.png" width="400">

Here're the `mcp_chatbot` code.

In [5]:
%%writefile mcp_project/mcp_chatbot.py
from dotenv import load_dotenv
from google import genai
from google.genai import types
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from typing import List
import asyncio
import nest_asyncio
import os

nest_asyncio.apply()
load_dotenv()

# Set the API key from environment variable
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', '')

class MCP_ChatBot:

    def __init__(self):
        # Initialize session and client objects
        self.session: ClientSession = None
        self.client = genai.Client()
        self.available_tools: List[dict] = []

    def build_function_declarations(self, mcp_tools):
        """Convert MCP tool metadata (JSON Schema) into Gemini FunctionDeclaration objects."""
        function_declarations = []
        for tool in mcp_tools:
            schema = tool.get("input_schema", {})
            if hasattr(schema, "model_dump"):
                schema = schema.model_dump()
            fd = types.FunctionDeclaration(
                name=tool["name"],
                description=tool.get("description", ""),
                parameters=schema or None,
            )
            function_declarations.append(fd)
        return function_declarations

    async def process_query(self, query):
        # Build Gemini tool configuration from available MCP tools
        function_declarations = self.build_function_declarations(self.available_tools)
        if function_declarations:
            genai_tool = types.Tool(function_declarations=function_declarations)
            config = types.GenerateContentConfig(tools=[genai_tool])
        else:
            config = types.GenerateContentConfig()

        response = self.client.models.generate_content(
            model='gemini-2.5-flash',
            contents=query,
            config=config
        )

        process_query = True
        while process_query:

            for part in response.candidates[0].content.parts:
                if hasattr(part, 'text') and part.text:
                    print(part.text)
                    process_query = False
                    break

                elif hasattr(part, 'function_call') and part.function_call:

                    function_call = part.function_call
                    tool_name = function_call.name
                    tool_args = dict(function_call.args)

                    print(f"Calling tool {tool_name} with args {tool_args}")

                    # Call tool via MCP session
                    result = await self.session.call_tool(tool_name, arguments=tool_args)

                    # Extract textual content from MCP tool result
                    result_text_parts = []
                    for result_part in getattr(result, "content", []) or []:
                        text_val = getattr(result_part, "text", None)
                        if text_val:
                            result_text_parts.append(text_val)
                    result_text = "\n".join(result_text_parts) if result_text_parts else str(getattr(result, "content", ""))

                    # Continue conversation with function response
                    function_response = types.FunctionResponse(
                        name=tool_name,
                        response={"result": result_text}
                    )

                    response = self.client.models.generate_content(
                        model='gemini-2.5-flash',
                        contents=[
                            types.Content(role='user', parts=[types.Part(text=query)]),
                            types.Content(role='model', parts=[types.Part(function_call=function_call)]),
                            types.Content(role='function', parts=[types.Part(function_response=function_response)])
                        ],
                        config=config
                    )

                    # Print the final response
                    if response.candidates[0].content.parts:
                        for final_part in response.candidates[0].content.parts:
                            if hasattr(final_part, 'text') and final_part.text:
                                print(final_part.text)
                                break
                    process_query = False

    async def chat_loop(self):
        """Run an interactive chat loop"""
        print("\nMCP Chatbot Started!")
        print("Type your queries or 'quit' to exit.")

        while True:
            try:
                query = input("\nQuery: ").strip()

                if query.lower() == 'quit':
                    break

                await self.process_query(query)
                print("\n")

            except Exception as e:
                print(f"\nError: {str(e)}")

    async def connect_to_server_and_run(self):
        # Create server parameters for stdio connection
        server_params = StdioServerParameters(
            command="uv",  # Executable
            args=["run", "research_server.py"],  # Optional command line arguments
            env=None,  # Optional environment variables
        )
        async with stdio_client(server_params) as (read, write):
            async with ClientSession(read, write) as session:
                self.session = session
                # Initialize the connection
                await session.initialize()

                # List available tools
                response = await session.list_tools()

                tools = response.tools
                print("\nConnected to server with tools:", [tool.name for tool in tools])

                self.available_tools = [{
                    "name": tool.name,
                    "description": tool.description,
                    "input_schema": tool.inputSchema
                } for tool in response.tools]

                await self.chat_loop()


async def main():
    chatbot = MCP_ChatBot()
    await chatbot.connect_to_server_and_run()


if __name__ == "__main__":
    asyncio.run(main())

Overwriting mcp_project/mcp_chatbot.py


### 수정된 Chatbot 코드 요약
- mcp_types.Tool 사용 제거: MCP 프로토콜의 Tool 모델( name, inputSchema 필수 ) 대신 Google Generative AI SDK의 `types.Tool` 사용
- MCP 서버 tool 메타데이터(JSON Schema)를 Gemini `types.FunctionDeclaration` 리스트로 변환하는 `build_function_declarations` 메서드 추가
- `process_query` 에서 function_declarations 존재 시 `types.Tool(function_declarations=[...])` 구성하여 `GenerateContentConfig` 에 전달
- Tool 실행 결과(`session.call_tool`)의 `result.content` 파트들에서 `.text` 추출 후 LLM function response로 전달
- 결과 문자열 합성 로직 추가(여러 part 대비)

이제 validation 오류(name, inputSchema missing)는 발생하지 않아야 하며, LLM이 function_call 을 생성하면 MCP 서버 tool이 호출되고 응답이 모델의 후속 출력에 반영됩니다.

## Running the MCP Chatbot

In [3]:
%%writefile mcp_project/main.py

def main():
    print("Hello from mcp-project!")


if __name__ == "__main__":
    main()


Writing mcp_project/main.py


In [4]:
%%writefile mcp_project/pyproject.toml
[project]
name = "mcp-project"
version = "0.1.0"
description = "Add your description here"
readme = "README.md"
requires-python = ">=3.11"
dependencies = []


Writing mcp_project/pyproject.toml


**Terminal Instructions**

In [3]:
# For local development, use your system terminal
print("To run the MCP chatbot locally:")
print("1. Open your system terminal")
print("2. Navigate to the mcp_project directory: cd mcp_project")
print("3. Activate virtual environment: source .venv/bin/activate")  
print("4. Install additional dependencies: uv add google-generativeai python-dotenv nest_asyncio")
print("5. Run the chatbot: uv run mcp_chatbot.py")

To run the MCP chatbot locally:
1. Open your system terminal
2. Navigate to the mcp_project directory: cd mcp_project
3. Activate virtual environment: source .venv/bin/activate
4. Install additional dependencies: uv add google-generativeai python-dotenv nest_asyncio
5. Run the chatbot: uv run mcp_chatbot.py
